# GPU 사용

## GPU 사용 확인

In [1]:
import torch
print(torch.cuda.is_available())  # True 반환 시 GPU 사용 가능
print(torch.version.cuda)  # 설치된 PyTorch가 지원하는 CUDA 버전 출력
print(torch.cuda.get_device_name(0))  # GPU 이름 출력

True
12.1
NVIDIA GeForce GTX 1650


## GPU 메모리 사용량 확인

In [2]:
print(f"총 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 3:.2f} GB")
print(f"사용 중인 메모리: {torch.cuda.memory_allocated(0) / 1024 ** 3:.2f} GB")
print(f"예약된 메모리: {torch.cuda.memory_reserved(0) / 1024 ** 3:.2f} GB")

총 메모리: 4.00 GB
사용 중인 메모리: 0.00 GB
예약된 메모리: 0.00 GB


# **1. 데이터 로드 및 기본정보 확인**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ruptures as rpt # 다양한 알고리즘을 제공하여 시계열 데이터에서 급격한 변화가 발생한 시점을 탐지

### **CSV 데이터 불러오기**

In [4]:
# 파일 경로
bio_data_path = './원본 데이터/생체 데이터.csv'
gyro_data_path = './원본 데이터/자이로 데이터.csv'

# 데이터 로드
bio_data = pd.read_csv(bio_data_path)
gyro_data = pd.read_csv(gyro_data_path)

# 데이터의 기본 정보
print("생체 데이터 정보:")
bio_data.info()
print("")
print("자이로 데이터 정보:")
gyro_data.info()

# 데이터 확인 결과 출력
print(f"\n생체 데이터 일부행:\n{bio_data.head()}\n\n자이로 데이터 일부 행:\n{gyro_data.head()}")

생체 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425992 entries, 0 to 425991
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   WorkDate     425992 non-null  int64  
 1   UserCode     425992 non-null  int64  
 2   Heartbeat    425992 non-null  int64  
 3   Temperature  425992 non-null  int64  
 4   Spo2         425992 non-null  int64  
 5   Latitude     425992 non-null  float64
 6   Longitude    425992 non-null  float64
 7   VitalDate    425992 non-null  object 
dtypes: float64(2), int64(5), object(1)
memory usage: 26.0+ MB

자이로 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920987 entries, 0 to 5920986
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   WorkDate      int64  
 1   UserCode      int64  
 2   X             float64
 3   Y             float64
 4   Z             float64
 5   RegisterDate  object 
dtypes: float64(3), int64(2), object(1)
memor

In [5]:
# 파일의 전체 행 수를 확인하는 방법
row_count = sum(1 for row in open(bio_data_path)) - 1  # 첫 번째 행은 헤더이므로 제외
print(f"생체 데이터 전체 행 수: {row_count}")

row_count = sum(1 for row in open(gyro_data_path)) - 1  # 첫 번째 행은 헤더이므로 제외
print(f"자이로 데이터 전체 행 수: {row_count}")

생체 데이터 전체 행 수: 425992
자이로 데이터 전체 행 수: 5920987


# **2. 데이터 전처리**

## **(1) 데이터 정리 (Data Cleaning)**

### **결측치 확인**
***
> 데이터에 결측치가 있는지 확인하고, 필요에 따라 결측치를 제거하거나 대체(imputation)

In [6]:
# 결측치 분석
bio_data_missing = bio_data.isnull().sum()
gyro_data_missing = gyro_data.isnull().sum()

print("생체 데이터 결측치:\n", bio_data_missing, "\n")
print("자이로 데이터 결측치:\n", gyro_data_missing)

생체 데이터 결측치:
 WorkDate       0
UserCode       0
Heartbeat      0
Temperature    0
Spo2           0
Latitude       0
Longitude      0
VitalDate      0
dtype: int64 

자이로 데이터 결측치:
 WorkDate        0
UserCode        0
X               0
Y               0
Z               0
RegisterDate    0
dtype: int64


### **결측치가 존재하는 경우**

In [7]:
# 결측치가 포함된 모든 행을 출력
missing_rows1 = bio_data[bio_data.isnull().any(axis=1)]
missing_rows2 = gyro_data[gyro_data.isnull().any(axis=1)]
print(f"{missing_rows1}\n {missing_rows2}\n")

# 각 열에 대한 결측치 비율을 확인
missing_percentage1 = gyro_data.isnull().mean() * 100
missing_percentage2 = gyro_data.isnull().mean() * 100
print(f"{missing_percentage1}\n {missing_percentage2}\n")

Empty DataFrame
Columns: [WorkDate, UserCode, Heartbeat, Temperature, Spo2, Latitude, Longitude, VitalDate]
Index: []
 Empty DataFrame
Columns: [WorkDate, UserCode, X, Y, Z, RegisterDate]
Index: []

WorkDate        0.0
UserCode        0.0
X               0.0
Y               0.0
Z               0.0
RegisterDate    0.0
dtype: float64
 WorkDate        0.0
UserCode        0.0
X               0.0
Y               0.0
Z               0.0
RegisterDate    0.0
dtype: float64



### **데이터 타입 변환**

In [8]:
# RegisterDate 열을 날짜 형식으로 변환합니다.
bio_data['VitalDate'] = pd.to_datetime(bio_data['VitalDate'])

# VitalDate 열을 날짜 형식으로 변환합니다.
gyro_data['RegisterDate'] = pd.to_datetime(gyro_data['RegisterDate'])

# 변환 결과를 확인합니다.
print(bio_data['VitalDate'].head(), "\n")
print(gyro_data['RegisterDate'].head())

0   2023-08-03 10:29:07.398
1   2023-08-03 10:29:08.442
2   2023-08-03 10:29:14.393
3   2023-08-03 10:29:16.379
4   2023-08-03 10:29:17.377
Name: VitalDate, dtype: datetime64[ns] 

0   2023-08-03 10:34:41
1   2023-08-03 10:32:47
2   2023-08-03 10:30:12
3   2023-08-03 10:33:41
4   2023-08-03 10:33:03
Name: RegisterDate, dtype: datetime64[ns]


> `VitalDate` 열의 데이터, `RegisterDate` 열의 데이터가 `문자열 형식`에서 `날짜 형식`으로 변환
> `pd.to_datetime()`: `VitalDate` 열, `RegisterDate` 열의 데이터를 `날짜 형식(datetime64[ns])`으로 변환

In [9]:
print("생체 데이터 정보:")
bio_data.info()

print("\n자이로 데이터 정보:")
gyro_data.info()

생체 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425992 entries, 0 to 425991
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   WorkDate     425992 non-null  int64         
 1   UserCode     425992 non-null  int64         
 2   Heartbeat    425992 non-null  int64         
 3   Temperature  425992 non-null  int64         
 4   Spo2         425992 non-null  int64         
 5   Latitude     425992 non-null  float64       
 6   Longitude    425992 non-null  float64       
 7   VitalDate    425992 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 26.0 MB

자이로 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920987 entries, 0 to 5920986
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   WorkDate      int64         
 1   UserCode      int64         
 2   X             float64       
 3   Y        

## **변화점 탐지(Change Pointer Detection)**

In [10]:
# 분석할 컬럼 선택
bio_columns = ['Heartbeat', 'Temperature', 'Spo2']
gyro_columns = ['X', 'Y', 'Z']

# 변화점 탐지 함수 정의
def detect_change_points(data, model="l2", pen=3):
    algo = rpt.Pelt(model=model).fit(data)
    return algo.predict(pen=pen)

# 배치 크기 설정
batch_size = 10000  # 예시: 한 번에 10,000개의 샘플을 처리

# 결과 저장 리스트
bio_change_points_all = {col: [] for col in bio_columns}
gyro_change_points_all = {col: [] for col in gyro_columns}

# 데이터 배치 처리 함수
def batch_change_point_detection(data, columns, batch_size, model="l2", pen=3):
    num_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)
    change_points_all = {col: [] for col in columns}
    
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        batch_data = data[start:end]
        
        for col in columns:
            change_points = detect_change_points(batch_data[col].values, model=model, pen=pen)
            # 배치의 인덱스 보정
            change_points = [cp + start for cp in change_points if cp + start < len(data)]
            change_points_all[col].extend(change_points)

    return change_points_all

# 생체 데이터 배치 처리
bio_change_points_all = batch_change_point_detection(bio_data, bio_columns, batch_size)

# 자이로 데이터 배치 처리
gyro_change_points_all = batch_change_point_detection(gyro_data, gyro_columns, batch_size)

# 결과 출력
print("Bio_Data Change Points:")
for col, points in bio_change_points_all.items():
    print(f"{col}: {points}")

print("\nGyro_Data Change Points:")
for col, points in gyro_change_points_all.items():
    print(f"{col}: {points}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### **변화점 탐지 결과 해석 및 기준 설정**
***
> 변화점 리스트: 각 변수에 대해 변화점이 감지된 시점의 인덱스가 리스트로 출력
>                예를 들어, `Heartbeat`에서 `[100, 250, 400]`과 같은 결과가 나온다면, 해당 인덱스에서
>                데이터의 패턴이 변화했다는 것을 의미

> 모델 및 패널티 설정: `penalty` 값을 조정하여 더 많은 또는 적은 변화점을 탐지 가능
>                      기본값으로 `3`을 사용했지만, 데이터를 기반으로 적절한 값을 실험적으로 조정해야 함
>                      모델은 `l2`, `rbf`, `linear`, `normal`, `ar` 등을 사용할 수 있습니다.

### **결과 시각화**

In [11]:
# 각 컬럼에 대해 변화점 시각화
def plot_change_points(data, change_points, title):
    plt.figure(figsize=(10, 6))
    plt.plot(data, label="Data")
    for cp in change_points:
        plt.axvline(x=cp, color='r', linestyle='--')
    plt.title(title)
    plt.legend()
    plt.show()

# Bio 데이터 시각화
for col in bio_columns:
    plot_change_points(bio_data[col].values, bio_change_points[col], f"Change Points in {col}")

# Gyro 데이터 시각화
for col in gyro_columns:
    plot_change_points(gyro_data[col].values, gyro_change_points[col], f"Change Points in {col}")

NameError: name 'bio_change_points' is not defined

### **기준 설정**
***
> 변화점 탐지 결과를 분석하여 적절한 기준을 설정
> 예를 들어, 특정 시간 동안 발생한 변화점의 빈도나 특정 변화가 발생하는 시점 등을 기준으로 정의

### 추가 조정
***
> 필요에 따라 패널티 값, 모델 유형 등을 조정하면서 적절한 변화를 포착할 수 있는 기준을 설정하고 이를 바탕으로 후속 분석을 진행
> 
> 이 과정으로 전체 데이터를 사용해 변화점 탐지를 수행하고, 적절한 기준을 설정 가능